In [1]:
using CSV;
using DataFrames;
using DelimitedFiles;
using LinearAlgebra;
using Plots
using Distributions

In [2]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/HSNIH-Palmer"

In [3]:
include("../../test/testHelper.jl");

In [4]:
results_BulkLMM = CSV.read("output/BulkLMM_results.csv", DataFrame; header = false) |> Matrix;

In [5]:
results_GEMMA = readdlm("output/gemma_results.txt.assoc.txt", '\t');

In [6]:
results_GEMMA

102280×10 Matrix{Any}:
   "chr"  "rs"                "ps"   "n_miss"  …   "l_mle"     "p_lrt"
 -9       "chr1:759319"     -9      0             0.0102581   0.0027058
 -9       "chr1:1154955"    -9      0             0.00803717  0.339739
 -9       "chr1:1649028"    -9      0             0.010183    0.00246692
 -9       "chr1:1649332"    -9      0             0.00981839  0.0112586
 -9       "chr1:1651967"    -9      0          …  0.0097179   0.0073924
 -9       "chr1:1671721"    -9      0             0.00985885  0.0102347
 -9       "chr1:1685077"    -9      0             0.00758233  0.338225
 -9       "chr1:1691966"    -9      0             0.0139525   0.0798655
 -9       "chr1:1754257"    -9      0             0.0147275   0.0463252
 -9       "chr1:1755622"    -9      0          …  0.00957821  0.00737652
 -9       "chr1:1779904"    -9      0             0.00596057  0.145273
 -9       "chr1:1782572"    -9      0             0.014643    0.045462
  ⋮                                         

In [7]:
using Distributions

In [8]:
function p2lod(pval::Float64, df::Int64)
    
    lrs = invlogcdf(Chisq(df), log(1-pval))
    lod = lrs/(2*log(10))

    return lod

end

p2lod (generic function with 1 method)

In [9]:
lods_gemma = map(x -> p2lod(x, 1), results_GEMMA[2:end, 10]);

In [16]:
results_BulkLMM[1:2, :]

2×1 Matrix{Float64}:
 127.51567136674777
   0.010559416521699195

In [10]:
lods_BulkLMM = results_BulkLMM[3:end, 1];

In [11]:
length(lods_BulkLMM)

102279

In [12]:
summary = hcat(lods_gemma, lods_BulkLMM);
summary

102279×2 Matrix{Float64}:
 1.95344   1.95344
 0.197911  0.197911
 1.99015   1.99015
 1.39496   1.39496
 1.55804   1.55804
 1.43178   1.43178
 0.199156  0.199156
 0.666129  0.666129
 0.862016  0.862016
 1.55888   1.55888
 0.460611  0.460611
 0.868897  0.868897
 0.250141  0.250141
 ⋮         
 0.105709  0.105709
 0.107674  0.107674
 0.302896  0.302896
 0.195024  0.195024
 0.210348  0.210348
 0.244478  0.244478
 0.211846  0.211846
 0.233952  0.233952
 0.161021  0.161021
 0.15539   0.15539
 0.131896  0.131896
 0.17615   0.17615

In [13]:
sum((lods_gemma .>= lods_BulkLMM) )/102279

0.49495986468385494

In [14]:
sum((lods_gemma .< lods_BulkLMM))/102279

0.505040135316145

In [15]:
sumSqDiff(reshape(lods_gemma, :, 1), reshape(lods_BulkLMM, :, 1))

1.20668182293601e-7